In [19]:
import pandas as pd
import numpy as np
import pyreadr
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support, roc_curve, roc_auc_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from typing import List, Optional, Sequence, Tuple, Union, Dict
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model as keras_load_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import label_binarize

from sklearn.manifold import TSNE, MDS
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn.metrics import classification_report, confusion_matrix, f1_score

import keras_tuner as kt
from dataclasses import dataclass, asdict
import random
from datetime import datetime
import joblib
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight






Step 1: We load the expression, gene and sample data we saved after the pre-processing we applied in R. We aim to predict tumor grade from the RNAseq data, so we examine potential class imbalance in the tumor grade sample metadata column, and we remove NA values in tumors with unidentified grade.

In [2]:
#load expression, genes and samples data
expression=pd.read_csv("C:/Users/joann/Desktop/M2/Deep_Learning/merged_expression.csv",index_col=0)
genes=pd.read_csv("C:/Users/joann/Desktop/M2/Deep_Learning/merged_genes.csv")
samples = pyreadr.read_r("C:/Users/joann/Desktop/M2/Deep_Learning/merged_samples.rds")
samples = samples[None]

In [3]:
#check what the expression, genes and samples data look like
print(expression.columns[:5])
print(expression.head(2))
print(samples.head(2))

Index(['ENSG00000276168.1', 'ENSG00000129824.16', 'ENSG00000133048.13',
       'ENSG00000012223.13', 'ENSG00000198695.2'],
      dtype='object')
                              ENSG00000276168.1  ENSG00000129824.16  \
TCGA-HT-7468-01A-11R-2027-07           8.423027           13.253145   
TCGA-DU-7015-01A-11R-2027-07           8.252623            5.532866   

                              ENSG00000133048.13  ENSG00000012223.13  \
TCGA-HT-7468-01A-11R-2027-07            7.491808            4.879747   
TCGA-DU-7015-01A-11R-2027-07            8.707316            5.087127   

                              ENSG00000198695.2  ENSG00000228253.1  \
TCGA-HT-7468-01A-11R-2027-07          15.950897          12.763416   
TCGA-DU-7015-01A-11R-2027-07          16.922914          13.431528   

                              ENSG00000198763.3  ENSG00000133110.15  \
TCGA-HT-7468-01A-11R-2027-07          17.025877            5.074539   
TCGA-DU-7015-01A-11R-2027-07          17.670450            4.085189   


In [4]:
#explore the samples metadata
print(samples.shape)
# list of column names
samples.columns.tolist()

#check for potential class imbalance in tumor grade
print(samples["tumor_grade"].value_counts())

#count how many NA values are in the tumor grade column
print(samples["tumor_grade"].isna().sum())

(925, 114)
tumor_grade
G4    391
G3    216
G2    211
Name: count, dtype: int64
107


In [5]:
#we exclude samples with NA tumor grade
samples_all = samples.copy() #keep original samples dataframe
expression_all = expression.copy() #keep original expression dataframe

samples_sup = samples_all.dropna(subset=["tumor_grade"])
expression_sup = expression_all.loc[samples_sup.index]

assert expression_sup.shape[0] == samples_sup.shape[0], "Mismatch in number of samples between expression and samples dataframes after dropping NA tumor grades."



Step 2: We configure the model and ensure reproducibility.

In [6]:
@dataclass
class Config:
    out_dir: str = "C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run"
    seed: int = 42

    # labels
    label_col: str = "tumor_grade"
    label_map: dict = None  # set in main if None

    # splits
    test_size: float = 0.15
    n_folds: int = 9  # 9-fold CV on train+val portion

    # training/tuning
    max_epochs: int = 100
    early_stop_patience: int = 5

    # tuner
    hyperband_factor: int = 3
    objective: str = "val_accuracy"

def set_global_seed(seed: int) -> None:
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # makes TF ops more deterministic when possible
    try:
        tf.config.experimental.enable_op_determinism()
    except Exception:
        pass

def make_run_dir(base_dir: str) -> str:
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(base_dir, ts)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

Step 3: We assign the classes. Grade II tumors will be 0, grade III will be 1 and grade IV will be 2.

In [7]:
def make_y(samples_sup: pd.DataFrame, label_col: str, label_map: dict) -> np.ndarray:
    """
    Convert tumor grade strings to integer classes.
    Example mapping: {"G2":0, "G3":1, "G4":2}
    """
    y = samples_sup[label_col].astype(str).map(label_map)
    if y.isna().any():
        bad = samples_sup.loc[y.isna(), label_col].unique()
        raise ValueError(f"Found unmapped labels: {bad}")
    return y.astype(int).to_numpy()


Step 4: We construct our  splits: we will have a hold-out test set, and run a 9-fold cross-validation on the rest of the data. In the cross-validation we include training and validation tests

In [8]:
def split_holdout_test(X: pd.DataFrame, y: np.ndarray, cfg: Config):
    """
    Hold out ONE stratified test set.
    Remaining data is used for 9-fold CV (train/val folds).
    """
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X, y,
        test_size=cfg.test_size,
        random_state=cfg.seed,
        stratify=y
    )
    return X_trainval, X_test, y_trainval, y_test

def save_test_ids(run_dir: str, X_test: pd.DataFrame) -> None:
    pd.Index(X_test.index).to_series().to_csv(os.path.join(run_dir, "test_ids.csv"), index=False)

def save_fold_ids(run_dir: str, fold: int, X_train: pd.DataFrame, X_val: pd.DataFrame) -> None:
    fold_dir = os.path.join(run_dir, f"fold_{fold:02d}")
    os.makedirs(fold_dir, exist_ok=True)
    pd.Index(X_train.index).to_series().to_csv(os.path.join(fold_dir, "train_ids.csv"), index=False)
    pd.Index(X_val.index).to_series().to_csv(os.path.join(fold_dir, "val_ids.csv"), index=False)

Step 5: We standardize the data, fitting on test data only to avoid data leakage

In [9]:
def standardize_train_val(X_train: pd.DataFrame, X_val: pd.DataFrame):
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_train_s = scaler.fit_transform(X_train).astype(np.float32)
    X_val_s = scaler.transform(X_val).astype(np.float32)
    return X_train_s, X_val_s, scaler

Step 6: We build our multi-layer perceptron model (MLP)

In [10]:
def build_model(hp: kt.HyperParameters, input_dim: int, num_classes: int = 3) -> keras.Model:
    """
    MLP with tunable depth and two architecture modes:
      1) Free: each layer can have its own units (picked from a grid)
      2) Pyramid: pick a base width; each next layer is half the previous (e.g. 32 -> 16 -> 8)
    """

    # batch size is tuned by your HyperbandWithBatchSize tuner
    hp.Choice("batch_size", [16, 32, 64, 128])

    # --- depth ---
    n_layers = hp.Int("n_layers", 2, 6)

    # --- regularization / optimizer ---
    l2_strength = hp.Choice("l2", [0.0, 1e-5, 1e-4, 1e-3])
    lr = hp.Choice("lr", [1e-4, 3e-4, 1e-3, 3e-3])

    # --- dropout (global) ---
    dropout = hp.Float("dropout", 0.1, 0.6, step=0.1)

    # --- units search space ---
    units_grid = [8, 16, 32, 64, 128, 256]

    # --- architecture mode ---
    arch_mode = hp.Choice("arch_mode", ["free", "pyramid_half"])

    # Decide units per layer
    layer_units = []

    if arch_mode == "free":
        for i in range(n_layers):
            u = hp.Choice(f"units_l{i+1}", units_grid)
            layer_units.append(u)

    elif arch_mode == "pyramid_half":
        base_units = hp.Choice("base_units", units_grid)
        min_units = hp.Choice("min_units", [8, 16])

        u = base_units
        for i in range(n_layers):
            layer_units.append(max(u, min_units))
            u = u // 2

    # --- build network ---
    inputs = keras.Input(shape=(input_dim,), name="expression")
    x = inputs

    for i, u in enumerate(layer_units, start=1):
        x = layers.Dense(
            units=u,
            activation="relu",
            kernel_regularizer=regularizers.l2(l2_strength),
            name=f"dense_{i}",
        )(x)

        if dropout and dropout > 0:
            x = layers.Dropout(dropout, name=f"dropout_{i}")(x)

    outputs = layers.Dense(num_classes, activation="softmax", name="softmax")(x)

    model = keras.Model(inputs, outputs, name="MLP_grade")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


Step 7: We tune the hyper-paramaters to choose the best model and avoid overfitting

In [11]:
class HyperbandWithBatchSize(kt.Hyperband):
    """Hyperband tuner that automatically uses hp['batch_size'] when fitting."""
    def run_trial(self, trial, *args, **kwargs):
        hp = trial.hyperparameters
        kwargs["batch_size"] = hp.get("batch_size")
        return super().run_trial(trial, *args, **kwargs)

def tune_hyperparameters(X_train_s, y_train, X_val_s, y_val, input_dim: int, run_dir: str, cfg: Config):
    tuner_dir = os.path.join(run_dir, "tuner")
    os.makedirs(tuner_dir, exist_ok=True)

    tuner = HyperbandWithBatchSize(
        hypermodel=lambda hp: build_model(hp, input_dim=input_dim, num_classes=3),
        objective=kt.Objective(cfg.objective, direction="max"),
        max_epochs=cfg.max_epochs,
        factor=cfg.hyperband_factor,
        directory=tuner_dir,
        project_name="grade_mlp"
    )

    early_stop = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        mode="max",
        patience=cfg.early_stop_patience,
        restore_best_weights=True
    )

    tuner.search(
        X_train_s, y_train,
        validation_data=(X_val_s, y_val),
        epochs=cfg.max_epochs,
        callbacks=[early_stop],
        verbose=1
    )

    best_hp = tuner.get_best_hyperparameters(1)[0]
    return tuner, best_hp

Step 8: We choose the best model to train

In [12]:
def make_class_weight(y_train: np.ndarray) -> dict:
    classes = np.unique(y_train)
    weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
    return {int(c): float(w) for c, w in zip(classes, weights)}

def train_best(X_train_s, y_train, X_val_s, y_val, input_dim: int, best_hp, run_dir: str, cfg: Config):
    model = build_model(best_hp, input_dim=input_dim, num_classes=3)

    ckpt_path = os.path.join(run_dir, "best_model.keras")

    early_stop = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        mode="max",
        patience=cfg.early_stop_patience,
        restore_best_weights=True
    )

    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=ckpt_path,
        monitor="val_loss",
        save_best_only=True
    )

    # class weights (computed on this fold's training labels)
    class_weight = make_class_weight(y_train)
    print("Class weight:", class_weight)

    history = model.fit(
        X_train_s, y_train,
        validation_data=(X_val_s, y_val),
        epochs=cfg.max_epochs,
        batch_size=best_hp.get("batch_size"),
        callbacks=[early_stop, checkpoint],
        verbose=1,
        class_weight=class_weight  # NEW
    )

    with open(os.path.join(run_dir, "history.json"), "w") as f:
        json.dump(history.history, f, indent=2)

    return model, ckpt_path

Step 9: We evaluate the model

In [13]:
def evaluate(model: keras.Model, X_eval_s, y_eval, run_dir: str, prefix: str = "eval"):
    eval_loss, eval_acc = model.evaluate(X_eval_s, y_eval, verbose=0)

    probs = model.predict(X_eval_s, verbose=0)
    y_pred = np.argmax(probs, axis=1)

    report = classification_report(y_eval, y_pred, output_dict=True)
    cm = confusion_matrix(y_eval, y_pred).tolist()
    macro_f1 = float(f1_score(y_eval, y_pred, average="macro"))

    results = {
        "loss": float(eval_loss),
        "accuracy": float(eval_acc),
        "macro_f1": macro_f1,
        "classification_report": report,
        "confusion_matrix": cm
    }

    with open(os.path.join(run_dir, f"{prefix}_metrics.json"), "w") as f:
        json.dump(results, f, indent=2)

    return results

Step 10: We save all the run metadata

In [14]:
def save_run_metadata(run_dir: str, cfg: Config, best_hp, scaler):
    with open(os.path.join(run_dir, "config.json"), "w") as f:
        json.dump(asdict(cfg), f, indent=2)

    with open(os.path.join(run_dir, "best_hyperparameters.json"), "w") as f:
        json.dump(best_hp.values, f, indent=2)

    joblib.dump(scaler, os.path.join(run_dir, "scaler.joblib"))

    versions = {
        "python": f"{os.sys.version_info.major}.{os.sys.version_info.minor}.{os.sys.version_info.micro}",
        "numpy": np.__version__,
        "pandas": pd.__version__,
        "tensorflow": tf.__version__,
        "keras_tuner": kt.__version__
    }
    with open(os.path.join(run_dir, "versions.json"), "w") as f:
        json.dump(versions, f, indent=2)

Step 11: We plot Evaluation Plots 

In [15]:
import os, json
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score,
)
from sklearn.preprocessing import label_binarize


def evaluate_and_save_plots(
    model,
    X_eval_s: np.ndarray,
    y_eval: np.ndarray,
    out_dir: str,
    class_names=("G2", "G3", "G4"),
    prefix: str = "test",
):
    os.makedirs(out_dir, exist_ok=True)
    n_classes = len(class_names)

    # ---- predictions ----
    eval_loss, eval_acc = model.evaluate(X_eval_s, y_eval, verbose=0)
    probs = model.predict(X_eval_s, verbose=0)          # (n, C)
    y_pred = np.argmax(probs, axis=1)

    # ---- metrics dict (same spirit as your evaluate()) ----
    report = classification_report(y_eval, y_pred, output_dict=True)
    cm = confusion_matrix(y_eval, y_pred)
    macro_f1 = float(f1_score(y_eval, y_pred, average="macro"))

    results = {
        "loss": float(eval_loss),
        "accuracy": float(eval_acc),
        "macro_f1": macro_f1,
        "classification_report": report,
        "confusion_matrix": cm.tolist(),
    }

    with open(os.path.join(out_dir, f"{prefix}_metrics.json"), "w") as f:
        json.dump(results, f, indent=2)

    # ----------------------------
    # 1) Confusion matrix (raw)
    # ----------------------------
    fig, ax = plt.subplots()
    im = ax.imshow(cm)
    ax.set_xticks(range(n_classes))
    ax.set_yticks(range(n_classes))
    ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_yticklabels(class_names)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(f"Confusion Matrix ({prefix})")
    for i in range(n_classes):
        for j in range(n_classes):
            ax.text(j, i, str(cm[i, j]), ha="center", va="center")
    fig.colorbar(im, ax=ax)
    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"{prefix}_confusion_matrix.png"), dpi=200)
    plt.close(fig)

    # ----------------------------
    # 2) Confusion matrix (row-normalized)
    # ----------------------------
    cm_norm = cm / np.clip(cm.sum(axis=1, keepdims=True), 1, None)
    fig, ax = plt.subplots()
    im = ax.imshow(cm_norm)
    ax.set_xticks(range(n_classes))
    ax.set_yticks(range(n_classes))
    ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_yticklabels(class_names)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(f"Confusion Matrix Normalized ({prefix})")
    for i in range(n_classes):
        for j in range(n_classes):
            ax.text(j, i, f"{cm_norm[i, j]:.2f}", ha="center", va="center")
    fig.colorbar(im, ax=ax)
    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"{prefix}_cm_normalized.png"), dpi=200)
    plt.close(fig)

    # ----------------------------
    # 3) Per-class precision/recall/F1 bar plot
    # ----------------------------
    rows = []
    for i, name in enumerate(class_names):
        d = report.get(str(i), None)
        if d is None:
            rows.append([name, np.nan, np.nan, np.nan])
        else:
            rows.append([name, d["precision"], d["recall"], d["f1-score"]])

    rows = np.array(rows, dtype=float, copy=False) if False else rows  # no-op; keeps notebook happy

    precision = [r[1] for r in rows]
    recall    = [r[2] for r in rows]
    f1s       = [r[3] for r in rows]

    fig, ax = plt.subplots()
    x = np.arange(n_classes)
    ax.bar(x - 0.2, precision, width=0.2, label="precision")
    ax.bar(x,       recall,    width=0.2, label="recall")
    ax.bar(x + 0.2, f1s,       width=0.2, label="f1")
    ax.set_xticks(x)
    ax.set_xticklabels(class_names)
    ax.set_ylim(0, 1.0)
    ax.set_title(f"Per-class metrics ({prefix})")
    ax.legend()
    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"{prefix}_per_class_metrics.png"), dpi=200)
    plt.close(fig)

    # ----------------------------
    # 4) ROC curves (one-vs-rest)
    # ----------------------------
    Y = label_binarize(y_eval, classes=list(range(n_classes)))  # (n, C)

    fpr, tpr, roc_auc = {}, {}, {}
    for c in range(n_classes):
        fpr[c], tpr[c], _ = roc_curve(Y[:, c], probs[:, c])
        roc_auc[c] = auc(fpr[c], tpr[c])

    fpr["micro"], tpr["micro"], _ = roc_curve(Y.ravel(), probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    fig, ax = plt.subplots()
    for c, name in enumerate(class_names):
        ax.plot(fpr[c], tpr[c], label=f"{name} (AUC={roc_auc[c]:.3f})")
    ax.plot(fpr["micro"], tpr["micro"], linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
    ax.plot([0, 1], [0, 1], linestyle=":")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(f"ROC (one-vs-rest) ({prefix})")
    ax.legend()
    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"{prefix}_roc_curves.png"), dpi=200)
    plt.close(fig)

    # ----------------------------
    # 5) Precision–Recall curves (one-vs-rest)
    # ----------------------------
    pr, re, ap = {}, {}, {}
    for c in range(n_classes):
        pr[c], re[c], _ = precision_recall_curve(Y[:, c], probs[:, c])
        ap[c] = average_precision_score(Y[:, c], probs[:, c])

    pr_micro, re_micro, _ = precision_recall_curve(Y.ravel(), probs.ravel())
    ap_micro = average_precision_score(Y, probs, average="micro")

    fig, ax = plt.subplots()
    for c, name in enumerate(class_names):
        ax.plot(re[c], pr[c], label=f"{name} (AP={ap[c]:.3f})")
    ax.plot(re_micro, pr_micro, linestyle="--", label=f"micro (AP={ap_micro:.3f})")
    ax.set_xlabel("Recall")
    ax.set_ylabel("Precision")
    ax.set_title(f"Precision–Recall (one-vs-rest) ({prefix})")
    ax.legend()
    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"{prefix}_pr_curves.png"), dpi=200)
    plt.close(fig)

    return results


Step 12: Permutation Feature Importance

In [16]:
def _predict_classes(model, X_s: np.ndarray, batch_size: int = 256) -> np.ndarray:
    probs = model.predict(X_s, verbose=0, batch_size=batch_size)
    return np.argmax(probs, axis=1)

def _compute_metrics(y_true, y_pred, n_classes: int):
    out = {}
    out["accuracy"] = accuracy_score(y_true, y_pred)
    out["macro_f1"] = f1_score(y_true, y_pred, average="macro")
    out["per_class_f1"] = f1_score(y_true, y_pred, average=None, labels=list(range(n_classes)))
    out["per_class_recall"] = recall_score(y_true, y_pred, average=None, labels=list(range(n_classes)))
    return out

def permutation_importance_multiclass(
    model,
    X_s: np.ndarray,
    y: np.ndarray,
    feature_names,
    out_dir: str,
    class_names=("G2", "G3", "G4"),
    n_repeats: int = 5,
    top_k: int = 10,
    max_features: int | None = 200,     # IMPORTANT: change to None to run all (can be very slow)
    batch_size_pred: int = 256,
    seed: int = 42,
):
    """
    Permutation importance on a fixed eval set (e.g. held-out test).

    Global metrics:
      - accuracy
      - macro_f1

    Per-class metrics:
      - f1 per class
      - recall per class (acts like per-class accuracy)

    Saves:
      - CSVs with mean +/- std drops
      - Top-K bar plots for each metric
    """
    os.makedirs(out_dir, exist_ok=True)
    rng = np.random.default_rng(seed)

    X_s = np.asarray(X_s)
    y = np.asarray(y).astype(int)
    n_classes = len(class_names)
    n_samples, n_features = X_s.shape

    feature_names = np.array(list(feature_names), dtype=object)
    if feature_names.shape[0] != n_features:
        raise ValueError("feature_names length does not match X_s number of columns.")

    # --- choose features to evaluate ---
    if (max_features is not None) and (n_features > max_features):
        # pick highest-variance features (simple, fast, reasonable default)
        variances = X_s.var(axis=0)
        feat_idx = np.argsort(-variances)[:max_features]
        feat_idx = np.sort(feat_idx)
        note = f"NOTE: computed permutation importance on top-{max_features} variance genes (out of {n_features})."
    else:
        feat_idx = np.arange(n_features)
        note = f"NOTE: computed permutation importance on ALL genes ({n_features})."

    with open(os.path.join(out_dir, "perm_importance_NOTE.txt"), "w") as f:
        f.write(note + "\n")
    print(note)

    # --- baseline predictions/metrics ---
    y_pred_base = _predict_classes(model, X_s, batch_size=batch_size_pred)
    base = _compute_metrics(y, y_pred_base, n_classes)

    # storage: drops (baseline - permuted) so higher drop => more important
    drops_acc = np.zeros((len(feat_idx), n_repeats), dtype=float)
    drops_macro_f1 = np.zeros((len(feat_idx), n_repeats), dtype=float)
    drops_f1 = np.zeros((len(feat_idx), n_repeats, n_classes), dtype=float)
    drops_recall = np.zeros((len(feat_idx), n_repeats, n_classes), dtype=float)

    X_work = X_s.copy()  # will permute columns in-place, then restore

    for fi_pos, j in enumerate(feat_idx):
        original_col = X_work[:, j].copy()

        for r in range(n_repeats):
            perm = rng.permutation(n_samples)
            X_work[:, j] = original_col[perm]

            y_pred_perm = _predict_classes(model, X_work, batch_size=batch_size_pred)
            m = _compute_metrics(y, y_pred_perm, n_classes)

            drops_acc[fi_pos, r] = base["accuracy"] - m["accuracy"]
            drops_macro_f1[fi_pos, r] = base["macro_f1"] - m["macro_f1"]
            drops_f1[fi_pos, r, :] = base["per_class_f1"] - m["per_class_f1"]
            drops_recall[fi_pos, r, :] = base["per_class_recall"] - m["per_class_recall"]

        # restore column
        X_work[:, j] = original_col

    # --- summarize ---
    names_sel = feature_names[feat_idx]

    df_global = pd.DataFrame({
        "feature": names_sel,
        "drop_accuracy_mean": drops_acc.mean(axis=1),
        "drop_accuracy_std": drops_acc.std(axis=1),
        "drop_macro_f1_mean": drops_macro_f1.mean(axis=1),
        "drop_macro_f1_std": drops_macro_f1.std(axis=1),
    }).sort_values("drop_macro_f1_mean", ascending=False)

    # per-class tables
    per_class_rows = []
    for c, cname in enumerate(class_names):
        tmp = pd.DataFrame({
            "feature": names_sel,
            f"drop_f1_{cname}_mean": drops_f1[:, :, c].mean(axis=1),
            f"drop_f1_{cname}_std": drops_f1[:, :, c].std(axis=1),
            f"drop_recall_{cname}_mean": drops_recall[:, :, c].mean(axis=1),
            f"drop_recall_{cname}_std": drops_recall[:, :, c].std(axis=1),
        })
        per_class_rows.append(tmp)

    df_per_class = per_class_rows[0]
    for k in per_class_rows[1:]:
        df_per_class = df_per_class.merge(k, on="feature")

    # save csvs
    df_global.to_csv(os.path.join(out_dir, "perm_importance_global.csv"), index=False)
    df_per_class.to_csv(os.path.join(out_dir, "perm_importance_per_class.csv"), index=False)

    # --- plotting helpers ---
    def _bar_topk(df, score_col, std_col, title, filename, top_k=10):
        d = df.sort_values(score_col, ascending=False).head(top_k).copy()
        d = d.iloc[::-1]  # so top is at top in barh

        fig, ax = plt.subplots(figsize=(8, 0.4*len(d) + 2.5))
        ax.barh(d["feature"].astype(str), d[score_col].to_numpy(), xerr=d[std_col].to_numpy())
        ax.set_xlabel("Performance drop after permutation")
        ax.set_title(title)
        fig.tight_layout()
        fig.savefig(os.path.join(out_dir, filename), dpi=200)
        plt.close(fig)

    # global plots
    _bar_topk(
        df_global, "drop_accuracy_mean", "drop_accuracy_std",
        title=f"Permutation importance (Global Accuracy) — Top {top_k}",
        filename=f"perm_top{top_k}_global_accuracy.png",
        top_k=top_k
    )
    _bar_topk(
        df_global, "drop_macro_f1_mean", "drop_macro_f1_std",
        title=f"Permutation importance (Global Macro-F1) — Top {top_k}",
        filename=f"perm_top{top_k}_global_macro_f1.png",
        top_k=top_k
    )

    # per-class plots (F1 + Recall)
    for cname in class_names:
        _bar_topk(
            df_per_class,
            f"drop_f1_{cname}_mean", f"drop_f1_{cname}_std",
            title=f"Permutation importance (Per-class F1: {cname}) — Top {top_k}",
            filename=f"perm_top{top_k}_f1_{cname}.png",
            top_k=top_k
        )
        _bar_topk(
            df_per_class,
            f"drop_recall_{cname}_mean", f"drop_recall_{cname}_std",
            title=f"Permutation importance (Per-class Recall: {cname}) — Top {top_k}",
            filename=f"perm_top{top_k}_recall_{cname}.png",
            top_k=top_k
        )

    # return dataframes + baseline metrics for reporting
    baseline_summary = {
        "baseline_accuracy": float(base["accuracy"]),
        "baseline_macro_f1": float(base["macro_f1"]),
        "baseline_per_class_f1": {class_names[i]: float(base["per_class_f1"][i]) for i in range(n_classes)},
        "baseline_per_class_recall": {class_names[i]: float(base["per_class_recall"][i]) for i in range(n_classes)},
        "note": note,
        "n_repeats": int(n_repeats),
        "evaluated_features": int(len(feat_idx)),
    }
    with open(os.path.join(out_dir, "perm_importance_baseline.json"), "w") as f:
        import json
        json.dump(baseline_summary, f, indent=2)

    print("Saved permutation importance CSVs + plots to:", out_dir)
    return df_global, df_per_class, baseline_summary

Final Step: Main run script

In [17]:
def main(expression_sup: pd.DataFrame, samples_sup: pd.DataFrame):
    cfg = Config()
    if cfg.label_map is None:
        cfg.label_map = {"G2": 0, "G3": 1, "G4": 2}

    set_global_seed(cfg.seed)
    run_dir = make_run_dir(cfg.out_dir)

    # 1) labels
    y = make_y(samples_sup, cfg.label_col, cfg.label_map)

    # 2) hold-out test
    X_trainval, X_test, y_trainval, y_test = split_holdout_test(expression_sup, y, cfg)
    save_test_ids(run_dir, X_test)

    # 3) 9-fold CV on trainval
    skf = StratifiedKFold(n_splits=cfg.n_folds, shuffle=True, random_state=cfg.seed)

    cv_results = []
    best_hp = None

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_trainval, y_trainval), start=1):
        fold_dir = os.path.join(run_dir, f"fold_{fold:02d}")
        os.makedirs(fold_dir, exist_ok=True)

        X_tr = X_trainval.iloc[tr_idx]
        X_va = X_trainval.iloc[va_idx]
        y_tr = y_trainval[tr_idx]
        y_va = y_trainval[va_idx]

        save_fold_ids(run_dir, fold, X_tr, X_va)

        # standardize per-fold 
        X_tr_s, X_va_s, _scaler_fold = standardize_train_val(X_tr, X_va)

        # tune once and reuse HP for all folds to save time
        if best_hp is None:
            tuner, best_hp = tune_hyperparameters(
                X_tr_s, y_tr,
                X_va_s, y_va,
                input_dim=X_tr_s.shape[1],
                run_dir=fold_dir,
                cfg=cfg
            )

            with open(os.path.join(run_dir, "best_hyperparameters.json"), "w") as f:
                json.dump(best_hp.values, f, indent=2)

        # train best HP on this fold
        model, ckpt_path = train_best(
            X_tr_s, y_tr,
            X_va_s, y_va,
            input_dim=X_tr_s.shape[1],
            best_hp=best_hp,
            run_dir=fold_dir,
            cfg=cfg
        )

        # evaluate on fold validation
        fold_metrics = evaluate(model, X_va_s, y_va, fold_dir, prefix="val")

        cv_results.append({
            "fold": fold,
            "val_accuracy": fold_metrics["accuracy"],
            "val_macro_f1": fold_metrics["macro_f1"],
            "model_path": ckpt_path
        })

    with open(os.path.join(run_dir, "cv_results.json"), "w") as f:
        json.dump(cv_results, f, indent=2)

    # 4) final training on all trainval with a small internal val split for early stopping
    X_tr_final, X_va_final, y_tr_final, y_va_final = train_test_split(
        X_trainval, y_trainval,
        test_size=0.15,
        random_state=cfg.seed,
        stratify=y_trainval
    )

    scaler_final = StandardScaler(with_mean=True, with_std=True)
    X_tr_final_s = scaler_final.fit_transform(X_tr_final).astype(np.float32)
    X_va_final_s = scaler_final.transform(X_va_final).astype(np.float32)
    X_test_s = scaler_final.transform(X_test).astype(np.float32)

    final_dir = os.path.join(run_dir, "final")
    os.makedirs(final_dir, exist_ok=True)

    model_final, ckpt_path_final = train_best(
        X_tr_final_s, y_tr_final,
        X_va_final_s, y_va_final,
        input_dim=X_tr_final_s.shape[1],
        best_hp=best_hp,
        run_dir=final_dir,
        cfg=cfg
    )

    #feature importance on final model
    df_global, df_per_class, baseline_pi = permutation_importance_multiclass(
    model=model_final,
    X_s=X_test_s,
    y=y_test,
    feature_names=X_test.columns,      # gene IDs in correct order
    out_dir=final_dir,                 # save next to your other plots
    class_names=("G2", "G3", "G4"),
    n_repeats=5,
    top_k=10,
    max_features=200,                  # IMPORTANT: increase if you can afford it
    batch_size_pred=256,
    seed=cfg.seed,
)

    # 5) evaluate once on held-out test
    class_names = ["G2", "G3", "G4"]
    test_results = evaluate_and_save_plots(
    model_final,
    X_test_s,
    y_test,
    out_dir=final_dir,     
    class_names=class_names,
    prefix="test",
    )
    # 6) save metadata + final scaler
    save_run_metadata(final_dir, cfg, best_hp, scaler_final)

    # print summary
    val_accs = [r["val_accuracy"] for r in cv_results]
    val_f1s = [r["val_macro_f1"] for r in cv_results]

    print("Saved run to:", run_dir)
    print("Final model:", ckpt_path_final)
    print(f"CV val accuracy: mean={np.mean(val_accs):.4f} std={np.std(val_accs):.4f}")
    print(f"CV val macro F1: mean={np.mean(val_f1s):.4f} std={np.std(val_f1s):.4f}")
    print("Test accuracy:", test_results["accuracy"])
    print("Test macro F1:", test_results["macro_f1"])

    return test_results, run_dir

In [20]:
#Now we call the main function to run the entire pipeline
results, run_dir = main(expression_sup, samples_sup)


Trial 254 Complete [00h 00m 17s]
val_accuracy: 0.7051281929016113

Best val_accuracy So Far: 0.8589743375778198
Total elapsed time: 00h 49m 16s
Class weight: {0: 1.2935010482180294, 1: 1.261758691206544, 2: 0.6971751412429379}
Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7293 - loss: 0.9049 - val_accuracy: 0.7179 - val_loss: 0.5916
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8298 - loss: 0.4919 - val_accuracy: 0.7949 - val_loss: 0.4891
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8460 - loss: 0.4206 - val_accuracy: 0.7564 - val_loss: 0.5025
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8768 - loss: 0.3406 - val_accuracy: 0.7436 - val_loss: 0.4952
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9141 - loss: 0.2731 - val_accuracy: 0.7821 - val_loss: 0.6887
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9173 - loss: 0.2539 - val_accuracy: 0.7692 - val_loss: 0.6804
Epoch 7/10

In [21]:
#look at the best model 
run_dir = r"C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937"
model_path = os.path.join(run_dir, "final", "best_model.keras")

model = load_model(model_path)
model.summary()

Model: "MLP_grade"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ expression (InputLayer)         │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       256,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 793,739 (3.03 MB)

 Trainable params: 264,579 (1.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 529,160 (2.02 MB)

In [22]:
hp_path = os.path.join(run_dir, "best_hyperparameters.json")
with open(hp_path, "r") as f:
    best_hp = json.load(f)

best_hp

{'batch_size': 16,
 'n_layers': 2,
 'l2': 1e-05,
 'lr': 0.001,
 'dropout': 0.1,
 'arch_mode': 'pyramid_half',
 'units_l1': 8,
 'units_l2': 8,
 'base_units': 128,
 'min_units': 8,
 'units_l3': 64,
 'units_l4': 16,
 'units_l5': 256,
 'units_l6': 256,
 'tuner/epochs': 12,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 4,
 'tuner/round': 2,
 'tuner/trial_id': '0116'}

In [23]:
#Functions to identify G2/G3 cluster errors

def load_ids_safely(path: str):
    df = pd.read_csv(path)
    ids = df.iloc[:, 0].astype(str).tolist()
    ids = [i for i in ids if i not in ("rownames", "index", "0", "None") and i == i]
    return ids

def analyze_g2g3_cluster_errors(
    run_dir: str,
    expression_sup: pd.DataFrame,
    samples_sup: pd.DataFrame,
    label_col: str = "tumor_grade",
    label_map: dict = None,
    cluster_cols: list = None,
):
    if label_map is None:
        label_map = {"G2": 0, "G3": 1, "G4": 2}

    if cluster_cols is None:
        cluster_cols = [
            "paper_Supervised.DNA.Methylation.Cluster",
            "paper_IDH.specific.RNA.Expression.Cluster",
            "paper_Pan.Glioma.DNA.Methylation.Cluster",
            "paper_IDH.specific.DNA.Methylation.Cluster",
        ]

    final_dir = os.path.join(run_dir, "final")
    model_path = os.path.join(final_dir, "best_model.keras")
    scaler_path = os.path.join(final_dir, "scaler.joblib")
    test_ids_path = os.path.join(run_dir, "test_ids.csv")

    model = load_model(model_path)
    scaler = joblib.load(scaler_path)

    # ---- id loading ----
    test_ids = load_ids_safely(test_ids_path)

    # keep only ids that exist in expression_sup
    test_ids = [i for i in test_ids if i in expression_sup.index]

    if len(test_ids) == 0:
        raise ValueError("No valid test IDs matched expression_sup.index. Check test_ids.csv format.")

    X_test = expression_sup.loc[test_ids]

    y_all = samples_sup[label_col].map(label_map)
    y_test = y_all.loc[X_test.index].astype(int).to_numpy()

    X_test_s = scaler.transform(X_test).astype(np.float32)
    probs = model.predict(X_test_s, verbose=0)
    y_pred = probs.argmax(axis=1)

    cluster_cols = [c for c in cluster_cols if c in samples_sup.columns]
    if len(cluster_cols) == 0:
        raise ValueError("None of the requested cluster_cols exist in samples_sup.")

    pred_df = pd.DataFrame({"y_true": y_test, "y_pred": y_pred}, index=X_test.index)
    pred_df = pred_df.join(samples_sup.loc[pred_df.index, cluster_cols])

    g23 = pred_df[pred_df["y_true"].isin([0, 1])].copy()
    g23["error_type"] = np.where(
        g23["y_true"] == g23["y_pred"],
        "correct",
        np.where((g23["y_true"] == 0) & (g23["y_pred"] == 1), "G2→G3",
        np.where((g23["y_true"] == 1) & (g23["y_pred"] == 0), "G3→G2", "other"))
    )
    g23["wrong_g2g3"] = g23["error_type"].isin(["G2→G3", "G3→G2"])

    print("G2/G3 confusion counts:")
    print(g23["error_type"].value_counts(dropna=False))

    for col in cluster_cols:
        s = g23[col].astype("string").fillna("NA")

        print("\n" + "="*80)
        print(f"Cluster column: {col}")
        print("="*80)

        counts = pd.crosstab(s, g23["wrong_g2g3"])
        print("\nCounts (False=correct, True=G2<->G3 confused):")
        print(counts)

        rates = pd.crosstab(s, g23["wrong_g2g3"], normalize="index")
        rates = rates.reindex(columns=[False, True], fill_value=0.0)
        rates.columns = ["correct_rate", "g2g3_confusion_rate"]



        print("\nPer-cluster rates:")
        print(rates)

        dir_tab = pd.crosstab(s, g23["error_type"])
        print("\nDirectional confusion:")
        print(dir_tab)

    return pred_df, g23


In [24]:
pred_df, g23_df = analyze_g2g3_cluster_errors(run_dir, expression_sup, samples_sup)


G2/G3 confusion counts:
error_type
correct    41
G3→G2      11
G2→G3       9
other       3
Name: count, dtype: int64

Cluster column: paper_Supervised.DNA.Methylation.Cluster

Counts (False=correct, True=G2<->G3 confused):
wrong_g2g3                                False  True 
paper_Supervised.DNA.Methylation.Cluster              
Classic-like                                  2      0
Codel                                        14      7
G-CIMP-high                                  20     12
Mesenchymal-like                              5      0
NA                                            1      0
PA-like                                       2      1

Per-cluster rates:
                                          correct_rate  g2g3_confusion_rate
paper_Supervised.DNA.Methylation.Cluster                                   
Classic-like                                  1.000000             0.000000
Codel                                         0.666667             0.333333
G-CIMP-high  

In [ ]:


run_dir = r"C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937"
fold_for_tuner = 1
project_name = "grade_mlp"

objective_name = "val_accuracy"
objective_direction = "max"

tuner_dir = os.path.join(run_dir, f"fold_{fold_for_tuner:02d}", "tuner")

# Assumes already defined in session:
# - HyperbandWithBatchSize
# - build_model
# - expression_sup 
input_dim = expression_sup.shape[1]

tuner = HyperbandWithBatchSize(
    hypermodel=lambda hp: build_model(hp, input_dim=input_dim, num_classes=3),
    objective=kt.Objective(objective_name, direction=objective_direction),
    max_epochs=100,   
    factor=3,         
    directory=tuner_dir,
    project_name=project_name,
    overwrite=False
)
tuner.reload()
print("Reloaded trials:", len(tuner.oracle.trials))


# Build dataframe of trials

rows = []
for trial_id, trial in tuner.oracle.trials.items():
    if trial.score is None:
        continue
    d = dict(trial.hyperparameters.values)
    d["trial_id"] = trial_id
    d["score"] = float(trial.score)  
    d["status"] = trial.status
    rows.append(d)

df = pd.DataFrame(rows)
if df.empty:
    raise ValueError("No trials with a score found. Check tuner_dir/project_name or objective_name.")

# stable order for plots
df = df.sort_values("trial_id").reset_index(drop=True)
df["trial_num"] = np.arange(1, len(df) + 1)

# derived columns
if "lr" in df.columns:
    df["log_lr"] = np.log10(df["lr"].astype(float))
if "l2" in df.columns:
    df["log_l2"] = np.where(df["l2"].astype(float) > 0, np.log10(df["l2"].astype(float)), np.nan)

def reconstruct_units(row):
    n_layers = int(row.get("n_layers", 0) or 0)
    if n_layers <= 0:
        return []

    # pyramid_half case
    if "base_units" in row and pd.notna(row.get("base_units")):
        base = int(row["base_units"])
        min_u = int(row.get("min_units", 8))
        u = base
        units = []
        for _ in range(n_layers):
            units.append(max(u, min_u))
            u //= 2
        return units

    # free case
    units = []
    for i in range(1, n_layers + 1):
        key = f"units_l{i}"
        if key in row and pd.notna(row.get(key)):
            units.append(int(row[key]))
    return units

df["width_max"] = df.apply(lambda r: max(reconstruct_units(r), default=np.nan), axis=1)
df["width_sum"] = df.apply(lambda r: np.sum(reconstruct_units(r)) if reconstruct_units(r) else np.nan, axis=1)


# 1) Histogram: trial val_accuracy distribution

scores = df["score"].to_numpy()
best_score = scores.max()  
mean_score = scores.mean()

fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(scores, bins=20)
ax.axvline(best_score, linestyle="--", linewidth=2, label=f"best = {best_score:.4f}")
ax.axvline(mean_score, linestyle=":", linewidth=2, label=f"mean = {mean_score:.4f}")
ax.set_xlabel(objective_name)
ax.set_ylabel("Count (trials)")
ax.set_title(f"Distribution of {objective_name} across trials")
ax.legend()
fig.tight_layout()

out_hist = os.path.join(run_dir, f"tuner_hist_{objective_name}.png")
fig.savefig(out_hist, dpi=200)
plt.close(fig)
print("Saved:", out_hist)


# 2) Bar plot: Spearman correlation with val_accuracy
# Positive rho => increasing HP tends to increase val_accuracy

hp_candidates = ["log_lr", "dropout", "log_l2", "n_layers", "batch_size", "base_units", "min_units", "width_max", "width_sum"]
hp_candidates = [c for c in hp_candidates if c in df.columns]

corrs = []
for c in hp_candidates:
    tmp = df[[c, "score"]].dropna()
    if len(tmp) < 3:
        continue
    rho = tmp[c].corr(tmp["score"], method="spearman")
    corrs.append((c, rho))

corrs = sorted(corrs, key=lambda x: abs(x[1]), reverse=True)

fig, ax = plt.subplots(figsize=(7, 4))
labels = [c for c, _ in corrs]
values = [rho for _, rho in corrs]
x = np.arange(len(labels))
ax.bar(x, values)
ax.axhline(0, linewidth=1)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_ylabel(f"Spearman ρ with {objective_name}")
ax.set_title(f"Hyperparameter association with {objective_name} (signed)")
fig.tight_layout()

out_corr = os.path.join(run_dir, f"tuner_corr_{objective_name}.png")
fig.savefig(out_corr, dpi=200)
plt.close(fig)
print("Saved:", out_corr)


# 3) Box plots: val_accuracy grouped by key HPs

def save_boxplot(group_col, out_name):
    tmp = df[[group_col, "score"]].dropna().copy()
    tmp[group_col] = tmp[group_col].astype(str)

    groups = []
    labels = []
    for lab, g in tmp.groupby(group_col):
        groups.append(g["score"].values)
        labels.append(lab)

    # sort labels numerically if possible
    try:
        order = np.argsort([float(x) for x in labels])
        labels = [labels[i] for i in order]
        groups = [groups[i] for i in order]
    except Exception:
        pass

    fig, ax = plt.subplots(figsize=(7, 4))
    ax.boxplot(groups, labels=labels)
    ax.set_xlabel(group_col)
    ax.set_ylabel(objective_name)
    ax.set_title(f"{objective_name} by {group_col}")
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
    fig.tight_layout()

    out_path = os.path.join(run_dir, out_name)
    fig.savefig(out_path, dpi=200)
    plt.close(fig)
    print("Saved:", out_path)

if "n_layers" in df.columns:
    save_boxplot("n_layers", f"tuner_box_n_layers_{objective_name}.png")
if "dropout" in df.columns:
    save_boxplot("dropout", f"tuner_box_dropout_{objective_name}.png")
if "batch_size" in df.columns:
    save_boxplot("batch_size", f"tuner_box_batch_size_{objective_name}.png")

if df["width_max"].notna().any():
    df["width_max_cat"] = df["width_max"].round().astype("Int64").astype(str)
    save_boxplot("width_max_cat", f"tuner_box_width_max_{objective_name}.png")





Reloading Tuner from C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937\fold_01\tuner\grade_mlp\tuner0.json
Reloaded trials: 254


c:\Users\joann\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Saved: C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937\tuner_hist_val_accuracy.png
Saved: C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937\tuner_corr_val_accuracy.png
Saved: C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937\tuner_box_n_layers_val_accuracy.png


C:\Users\joann\AppData\Local\Temp\ipykernel_2760\1270215146.py:160: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(groups, labels=labels)
C:\Users\joann\AppData\Local\Temp\ipykernel_2760\1270215146.py:160: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(groups, labels=labels)


Saved: C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937\tuner_box_dropout_val_accuracy.png
Saved: C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937\tuner_box_batch_size_val_accuracy.png


C:\Users\joann\AppData\Local\Temp\ipykernel_2760\1270215146.py:160: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(groups, labels=labels)


Saved: C:/Users/joann/Desktop/M2/Deep_Learning/MLP_run/20260105_025937\tuner_box_width_max_val_accuracy.png


C:\Users\joann\AppData\Local\Temp\ipykernel_2760\1270215146.py:160: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(groups, labels=labels)
